In [ ]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.4/74.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.5/443.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 20.6 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.99.1
    Uninstalling openai-1.99.1:
      Successfully uninstalled openai-1.99.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.72
    Uninstalling langchain-core-0.3.72:
      Successfully uninstalled langchain-core-0.3.72


In [ ]:
import os
import json
import re
from typing import List
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

class EvaluationResult(BaseModel):
    relevance: float = Field(..., description="Relevance score between 0 and 100")
    factual_accuracy: float = Field(..., description="Accuracy score between 0 and 100")
    completeness: float = Field(..., description="Completeness score between 0 and 100")
    overall_score: float = Field(..., description="Weighted score between 0 and 100")

llm = ChatOpenAI(model="o4-mini")

prompt_template = PromptTemplate(
    input_variables=["question", "references", "student_answer"],
    template="""
You are an evaluator. Compare the STUDENT ANSWER to the REFERENCE if reference is available or else evaluate yourself.

Question: {question}

Reference Answers:
{references}

Student Answer:
{student_answer}

Scoring rules:
- relevance: between 0 and 100
- factual_accuracy: between 0 and 100
- completeness: between 0 and 100
Formula: overall_score = 0.35 * factual_accuracy + 0.45 * relevance + 0.2 * completeness

Respond ONLY with valid JSON in this format:
{{
  "relevance": <float>,
  "factual_accuracy": <float>,
  "completeness": <float>,
  "overall_score": <float>
}}
"""
)

def extract_json(text: str) -> dict:
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if not match:
        raise ValueError(f"No JSON found in model output: {text}")
    json_str = match.group(0)
    return json.loads(json_str)

def evaluate(question: str, references: List[str], student_answer: str) -> dict:
    prompt = prompt_template.format(
        question=question,
        references="\n".join(references),
        student_answer=student_answer
    )
    response = llm.invoke(prompt)
    response_text = response.content if hasattr(response, "content") else str(response)
    result_dict = extract_json(response_text)
    validated = EvaluationResult(**result_dict)
    return validated.dict()

In [ ]:
q = "What is p-value in statistics?"

refs = ["The p-value is a fundamental concept in statistical hypothesis testing, used to quantify the strength of evidence against a null hypothesis. It is defined as the probability of obtaining a test statistic at least as extreme as the one actually observed, assuming the null hypothesis is true. In other words, it answers the question: “If the null hypothesis were correct, how likely is it that we would see results like this (or more extreme) purely by random chance?"
"A small p-value (commonly less than a chosen significance level, such as 0.05) suggests that the observed data would be quite rare if the null hypothesis were true. This provides evidence in favor of the alternative hypothesis, leading researchers to reject the null. Conversely, a large p-value indicates that the observed data is consistent with the null hypothesis, and there is insufficient evidence to reject it. Importantly, the p-value does not measure the probability that the null hypothesis itself is true, nor does it indicate the size or practical importance of an effect — it only assesses the compatibility between the data and the null model."
"In practice, p-values are used alongside effect sizes, confidence intervals, and subject-matter knowledge to make informed conclusions. While widely used, p-values can be misinterpreted and should not be the sole basis for decision-making; they are best understood as one piece of the broader statistical inference process."]

s0 = "p-value is probability"
s1 = "Area under the distribution curve outside the confidence interval is the p-value"
s2 = "Probability of an event that is atleast as extreme as the observed event is known as p-value"
s3 = "p-value measures the probability of an event that is atleast as extreme as the observed event, assuming null hypothesis is true"
s4 = "The p-value is a measure of how compatible your observed data is with the assumption that the null hypothesis is correct — it represents the likelihood of seeing the observed effect or something more extreme purely due to random chance if the null hypothesis holds."

student = [s0, s1, s2, s3, s4]

for i in range(len(student)):
  print(json.dumps(evaluate(q, refs, student[i]), indent=2))

/tmp/ipython-input-43586821.py:66: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return validated.dict()


{
  "relevance": 30.0,
  "factual_accuracy": 40.0,
  "completeness": 5.0,
  "overall_score": 28.5
}


/tmp/ipython-input-43586821.py:66: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return validated.dict()


{
  "relevance": 40.0,
  "factual_accuracy": 10.0,
  "completeness": 10.0,
  "overall_score": 23.5
}


/tmp/ipython-input-43586821.py:66: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return validated.dict()


{
  "relevance": 90.0,
  "factual_accuracy": 80.0,
  "completeness": 20.0,
  "overall_score": 72.5
}


/tmp/ipython-input-43586821.py:66: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return validated.dict()


{
  "relevance": 100.0,
  "factual_accuracy": 95.0,
  "completeness": 40.0,
  "overall_score": 86.25
}
{
  "relevance": 100.0,
  "factual_accuracy": 95.0,
  "completeness": 75.0,
  "overall_score": 93.25
}


/tmp/ipython-input-43586821.py:66: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  return validated.dict()


In [ ]:
# Quick Test of Automated Evaluation
import json

# Load the response file
with open("response_1755348961176.json", 'r', encoding='utf-8') as file:
    data = json.load(file)

print(f"Loaded {len(data['results'])} questions from the response file")

# Test evaluation on first question with available reference answers
test_results = []
questions_processed = 0

for idx, result in enumerate(data['results'][:3]):  # Test first 3 questions only
    question = result['question']
    domain = result.get('domain', 'Unknown')
    difficulty = result.get('difficulty', 'Unknown')
    
    # Check if reference answers exist
    reference_answers = []
    if 'reference_answers' in result and result['reference_answers']:
        for ref in result['reference_answers']:
            if isinstance(ref, dict) and 'answer' in ref:
                reference_answers.append(ref['answer'])
    
    if not reference_answers:
        print(f"Skipping question {idx+1}: No reference answers")
        continue
    
    print(f"\nProcessing Question {idx+1}:")
    print(f"Domain: {domain}, Difficulty: {difficulty}")
    print(f"Question: {question[:100]}...")
    
    # Test with first candidate answer if available
    if 'candidate_answers' in result and result['candidate_answers']:
        candidate_answer = result['candidate_answers'][0]
        if isinstance(candidate_answer, dict):
            answer_text = candidate_answer.get('answer', '')
            answer_type = candidate_answer.get('type', 'unknown')
        else:
            answer_text = candidate_answer
            answer_type = 'unknown'
        
        print(f"Evaluating answer type: {answer_type}")
        print(f"Answer preview: {answer_text[:150]}...")
        
        try:
            # Use the existing evaluate function
            evaluation = evaluate(question, reference_answers, answer_text)
            evaluation['question_index'] = idx + 1
            evaluation['domain'] = domain
            evaluation['difficulty'] = difficulty
            evaluation['answer_type'] = answer_type
            
            test_results.append(evaluation)
            questions_processed += 1
            
            print(f"✅ Evaluation completed:")
            print(f"   Overall Score: {evaluation['overall_score']:.1f}")
            print(f"   Relevance: {evaluation['relevance']:.1f}")
            print(f"   Accuracy: {evaluation['factual_accuracy']:.1f}")
            print(f"   Completeness: {evaluation['completeness']:.1f}")
            
        except Exception as e:
            print(f"❌ Evaluation failed: {e}")

print(f"\n{'='*50}")
print(f"TEST EVALUATION SUMMARY")
print(f"{'='*50}")
print(f"Questions processed: {questions_processed}")

if test_results:
    avg_overall = sum(r['overall_score'] for r in test_results) / len(test_results)
    avg_relevance = sum(r['relevance'] for r in test_results) / len(test_results)
    avg_accuracy = sum(r['factual_accuracy'] for r in test_results) / len(test_results)
    avg_completeness = sum(r['completeness'] for r in test_results) / len(test_results)
    
    print(f"Average Overall Score: {avg_overall:.2f}")
    print(f"Average Relevance: {avg_relevance:.2f}")
    print(f"Average Factual Accuracy: {avg_accuracy:.2f}")
    print(f"Average Completeness: {avg_completeness:.2f}")
    
    # Show individual results
    print(f"\nDetailed Results:")
    for i, result in enumerate(test_results, 1):
        print(f"  {i}. Domain: {result['domain']}, Score: {result['overall_score']:.1f}")

print(f"\n✅ Automated evaluation system is working!")
print(f"💡 To evaluate all questions, run the full evaluation_runner.py script")